**TASK - 1**

**Parsing Text Files**


Regular expressions are expressions that checks if a set of strings match it. Every string has a regular expression and by using them, you can see if the text matches a particular string. Regular expressions are used as the term "re" in python. In this task, we shall see how regular expressions can help us retrieve only the required strings from the the documents.



Importing all the required libraries which are re and os

In [12]:
from google.colab import drive
import os
import re

XML files are used to store the data in a particular order using the help of tags. By this we can define in what order we want to store the data and passed over the internet. In this case, the username, latest review date, product ID, review date, review helpful, review text and the review summary. The resulting XML file can be opened through a web browser application or a text file depending on what format yu want it to be.



Defining the required Body structures for the XML file that is to be generated. The asked format is:


*   users : for all the users
*   user: for all the reviews of a particular user that includes latest review date, and username 
*   reviews: for all the reviews of a specific user
*   review: for each user's product ID, review date, helpful, text, and summary of the user review



In [13]:

xml_body = '''<users>
{}
</users>'''

user_body = '''<user name = "{}">
<latest_review_date>{}</latest_review_date>
<reviews>
{}
</reviews>
</user>'''

review_body = '''<review> 
<productID>{}</productID>
<review_date>{}</review_date>
<review_helpful>{}</review_helpful>
<review_text>{}</review_text>
<review_summary>{}</review_summary>
</review>'''

Here, the XML body structure is defined as how we want the end result file to look like. The next step is to make changes to the data so as to write it into an XML file.

The function 'userdata' is for cleaning the data into the required format. In the given input file, each subheading of the review like review_id, product_id, etc are starting with a "\$" symbol. So in order to split the texts, we use '$' to split the data by which we get the categories separately.
Next step is to use the re for the Review block which includes variables - rev_id, product_id, review_date, helpful, review_text, review_summary. Based on this, we split the variables and contents further into separate entities as the variable and its corresponding context.
One task that was mentioned was to handle the special characters. So we use 're' to match special characters and then replace it accordingly. 
Finally, the function retruns the rev_id, review_main_body and review_date for the review block of the XML file.

In [14]:
def userdata(main_data):
#splitting the phrase with respect to '$' 
  product_info = main_data.split("$")
  product_info[:] = [i for i in product_info if i != '']

#using re for the review block which includes rev_id, product_id, review_date, helpful, review_text, review_summary

  for info in product_info:
    if re.match("rev\w*[_]*[\.]*ID", info, flags = re.IGNORECASE):
      rev_id = info.split(":")[1].strip()
    elif re.match("prod\w*[_]*[\.]*ID", info, flags = re.IGNORECASE):
      product_id = info.split(":",1)[1].strip()
    elif re.match("\w*date", info, flags = re.IGNORECASE):
      review_date = info.split(":", 1)[1].strip()
    elif re.match("\w*\.*[ ]*help", info, flags = re.IGNORECASE):
      review_helpful = info.split(":",1)[1].strip()
      
      #handling special characters in the data
    elif re.match("\w*[_]*[ ]*summary", info, flags = re.IGNORECASE):
      review_summary = info.split(":",1)[1].strip().replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('\n', '')
    elif re.match("REVIEW", info, flags = re.IGNORECASE):
      review_text = info.split(":",1)[1].strip().replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('\n', '')

 
      
  review_main_body = review_body.format(product_id, review_date, review_helpful, review_text, review_summary)
  return rev_id, review_main_body, review_date


We are creating a file in which we will be writing the contents of the input text. 

In [15]:
def writexml(xml_all_user):
  with open(location1, 'w') as f:
    f.write(xml_body.format(xml_all_user))
    f.close()

Mounting the drive to the colab to access the data from the drive


In [ ]:

#mounting the drive to access the contents of the drive
drive.mount('/content/drive/')

To access the data from the drive, the variable "location" has been defined. The variable "location" has the location of the input data  

In [17]:
location = "/content/drive/Shareddrives/FIT5196-s1-2022/A1/Task1/input_data/32263546/"

The location1 variable shows where the XML file should be written

In [18]:
location1 = "/content/drive/MyDrive/dw_assignment1_output/32263546.xml"

Accessing all the 40 files inside the input folder of the student




In [ ]:
#printing all the files inside the input folder
l = os.path.dirname(location)
list_of_files = os.listdir(l)
print(list_of_files)
print(type(list_of_files))

Reading all the files from the student input folder and appending it to a variable named 'data'.

In [20]:
data = ""
for files in list_of_files:
    f = open(location + files)
    data = data + f.read()
    f.close()


Splitting the data into required format  and making it into a list

In [21]:
#splitting the data into the required chunks 
data_list = re.split('(\$rev\w*[_]*[\.]*ID)',  data)
data_list[:] = [i for i in data_list if i != '']

An empty dictionary 'xml_dict' is made for the reviews of each user. Since one user can have multiple reviews, this dictionary stores all the reviews of each user. The key of the dictionary will be the username and the value will have all the reviews given by that key user. The reviews are passed through the usersata function to get cleaned review of desirable format. There can also be new reviews for the user. So appending those reviews into the corresponding user's values. 
Although there are other ways to do this, re and os has been used because of the library criteria mentioned.
Once this is done, values of the latest review date is retrieved. The retrieved date is checked for the year. If the date in the dictionary is greater than the newly retrieved date, pass. If the year in the dictionary  is less than the one in the newly retrieved data, then the newly retrieved date is assigned to the one in the dictionary. But if the year is equal, then the month is checked. 
Finally, the dictionary is passed into a specific format so as to write it into an XML file. 


In [22]:
#creating a dictionary for the reviews for each user as there can be multiple reviews for a user. key = username, value = list of reviews
xml_dict = {}
for i in range(0, len(data_list)-1):

  #passing the value though userdata to get cleaned value of each review
  if(i%2 == 0):
    main_data = data_list[i] + data_list[i+1]
    user, xml_review, xml_date = userdata(main_data)

    #appending the new reviews into the existing reviews of the corresponding user
    if user in xml_dict:
      xml_dict[user]["reviews"].append(xml_review)
    else:
      xml_dict[user] = {"reviews": [xml_review], "latest_date":xml_date}
    
    #retrieveing the values of the latest review date 
    #checking the year
    if(int(xml_dict[user]["latest_date"].split(" ")[-1]) > int(xml_date.split(" ")[-1])):
      pass
    elif(int(xml_dict[user]["latest_date"].split(" ")[-1]) == int(xml_date.split(" ")[-1])):
      if(int(xml_dict[user]["latest_date"].split(" ")[0]) > int(xml_date.split(" ")[0])):
        pass
      if(int(xml_dict[user]["latest_date"].split(" ")[0]) < int(xml_date.split(" ")[0])):
        xml_dict[user]["latest_date"] = xml_date
    else:
      xml_dict[user]["latest_date"] = xml_date

#passing the dcitionary into a specific format to write it to a file.
xml_all_user = ""
for user, user_dict in xml_dict.items():
  xml_all_user = xml_all_user + "\n" + user_body.format(user, user_dict["latest_date"], "\n".join(user_dict["reviews"]))

writexml(xml_all_user.strip())

Thus the text files are parsed and the desired XML file has been generated using re and os.